# Governor's Proposal - Statewide Impact Analysis (RI-2 Dataset)

This notebook calculates the statewide impact of the Governor's RI CTC proposal using the RI-2 dataset.

In [9]:
from policyengine_us import Microsimulation
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '.')

from ri_ctc_calc.calculations.reforms import create_custom_reform

RI_DATASET = "hf://policyengine/test/RI-2.h5"
YEAR = 2027

In [10]:
# Governor's Proposal Parameters
reform_params = {
    'ctc_amount': 325,
    'ctc_age_limit': 19,
    'ctc_refundability_cap': 100000,
    'ctc_phaseout_rate': 0,
    'ctc_phaseout_thresholds': {
        'SINGLE': 0,
        'JOINT': 0,
        'HEAD_OF_HOUSEHOLD': 0,
        'SURVIVING_SPOUSE': 0,
        'SEPARATE': 0,
    },
    'ctc_stepped_phaseout': True,
    'ctc_stepped_phaseout_threshold': 265965,
    'ctc_stepped_phaseout_increment': 7590,
    'ctc_stepped_phaseout_rate_per_step': 0.20,
    'ctc_young_child_boost_amount': 0,
    'ctc_young_child_boost_age_limit': 6,
    'enable_exemption_reform': True,
    'exemption_amount': 0,
    'exemption_age_limit_enabled': True,
    'exemption_age_threshold': 19,
    'exemption_phaseout_rate': 0,
    'exemption_phaseout_thresholds': {
        'SINGLE': 0,
        'JOINT': 0,
        'HEAD_OF_HOUSEHOLD': 0,
        'SURVIVING_SPOUSE': 0,
        'SEPARATE': 0,
    },
    'year': YEAR
}

print("Governor's Proposal Parameters:")
print(f"  CTC Amount: ${reform_params['ctc_amount']}/child")
print(f"  Age Limit: {reform_params['ctc_age_limit']}")
print(f"  Stepped Phaseout: {reform_params['ctc_stepped_phaseout']}")
print(f"  Phaseout Threshold: ${reform_params['ctc_stepped_phaseout_threshold']:,}")
print(f"  Phaseout Rate: {reform_params['ctc_stepped_phaseout_rate_per_step']*100}% per ${reform_params['ctc_stepped_phaseout_increment']:,}")
print(f"  Exemption Reform: Zeroes dependent exemption for children")

Governor's Proposal Parameters:
  CTC Amount: $325/child
  Age Limit: 19
  Stepped Phaseout: True
  Phaseout Threshold: $265,965
  Phaseout Rate: 20.0% per $7,590
  Exemption Reform: Zeroes dependent exemption for children


In [11]:
# Load simulations
print(f"Loading RI-2 dataset: {RI_DATASET}")
sim_baseline = Microsimulation(dataset=RI_DATASET)

reform = create_custom_reform(**reform_params)
sim_reform = Microsimulation(dataset=RI_DATASET, reform=reform)
print("Simulations loaded.")

Loading RI-2 dataset: hf://policyengine/test/RI-2.h5
Simulations loaded.


In [12]:
# Calculate household net income for both scenarios
net_income_baseline = sim_baseline.calculate("household_net_income", period=YEAR, map_to="household")
net_income_reform = sim_reform.calculate("household_net_income", period=YEAR, map_to="household")

# Calculate the benefit (increase in net income)
net_income_change = net_income_reform - net_income_baseline

# Get weights and other data
household_weight = sim_reform.calculate("household_weight", period=YEAR)
agi = sim_reform.calculate("adjusted_gross_income", period=YEAR, map_to="household")
eligible_children = sim_reform.calculate("ri_ctc_eligible_children", period=YEAR, map_to="household")

In [13]:
# Aggregate statistics (matching microsimulation.py logic)
# MicroSeries already has weights built in, so .sum() and .mean() are weighted

# Total cost - weighted sum from MicroSeries
total_cost = net_income_change.sum()

# Count affected households
affected_households = (np.abs(net_income_change) > 1).sum()
beneficiaries_mask = net_income_change > 0
beneficiaries = beneficiaries_mask.sum()  # Unweighted count of sample units

# Average benefit - weighted mean from MicroSeries
avg_benefit = net_income_change[np.abs(net_income_change) > 1].mean() if affected_households > 0 else 0

# Children affected - weighted sum
children_affected = eligible_children[beneficiaries_mask].sum() if beneficiaries > 0 else 0

# Total households (weighted)
total_households = household_weight.sum()

# Winners/losers analysis - use household weights for population-level counts
winners_mask = net_income_change > 1
losers_mask = net_income_change < -1
winners = household_weight[winners_mask].sum()  # Weighted count
losers = household_weight[losers_mask].sum()  # Weighted count
winners_rate = (winners / total_households * 100) if total_households > 0 else 0
losers_rate = (losers / total_households * 100) if total_households > 0 else 0

print("="*70)
print("STATEWIDE IMPACT - GOVERNOR'S PROPOSAL (RI-2 Dataset)")
print("="*70)
print(f"Total Cost: ${total_cost:,.0f}")
print(f"Beneficiary Households (sample): {beneficiaries:,.0f}")
print(f"Average Benefit: ${avg_benefit:,.0f}")
print(f"Children Affected: {children_affected:,.0f}")
print(f"")
print(f"Winners: {winners:,.0f} ({winners_rate:.1f}%)")
print(f"Losers: {losers:,.0f} ({losers_rate:.1f}%)")
print(f"Total Households: {total_households:,.0f}")
print("="*70)

STATEWIDE IMPACT - GOVERNOR'S PROPOSAL (RI-2 Dataset)
Total Cost: $39,879,203
Beneficiary Households (sample): 120,966
Average Benefit: $330
Children Affected: 221,871

Winners: 7,144,727 (31.3%)
Losers: 0 (0.0%)
Total Households: 22,795,617


In [14]:
# Poverty impact (using person_in_poverty variable at person level)
total_population = sim_baseline.calculate("person_count", period=YEAR).sum()

person_in_poverty_baseline = sim_baseline.calculate("person_in_poverty", period=YEAR)
person_in_poverty_reform = sim_reform.calculate("person_in_poverty", period=YEAR)

poverty_baseline_count = person_in_poverty_baseline.sum()
poverty_reform_count = person_in_poverty_reform.sum()
poverty_baseline_rate = (poverty_baseline_count / total_population * 100) if total_population > 0 else 0
poverty_reform_rate = (poverty_reform_count / total_population * 100) if total_population > 0 else 0
poverty_rate_change = poverty_reform_rate - poverty_baseline_rate  # Percentage point change
# Percent change (relative): ((new - old) / old) * 100
poverty_percent_change = ((poverty_reform_rate - poverty_baseline_rate) / poverty_baseline_rate * 100) if poverty_baseline_rate > 0 else 0

# Child poverty impact (filter poverty to children only)
is_child = sim_baseline.calculate("is_child", period=YEAR)
total_children = is_child.sum()

child_poverty_baseline = (person_in_poverty_baseline & is_child).sum()
child_poverty_reform = (person_in_poverty_reform & is_child).sum()
child_poverty_baseline_rate = (child_poverty_baseline / total_children * 100) if total_children > 0 else 0
child_poverty_reform_rate = (child_poverty_reform / total_children * 100) if total_children > 0 else 0
child_poverty_rate_change = child_poverty_reform_rate - child_poverty_baseline_rate  # Percentage point change
# Percent change (relative): ((new - old) / old) * 100
child_poverty_percent_change = ((child_poverty_reform_rate - child_poverty_baseline_rate) / child_poverty_baseline_rate * 100) if child_poverty_baseline_rate > 0 else 0

print("\n" + "="*70)
print("POVERTY IMPACT")
print("="*70)
print(f"Overall Poverty Rate:")
print(f"  Baseline: {poverty_baseline_rate:.2f}%")
print(f"  Reform: {poverty_reform_rate:.2f}%")
print(f"  Change: {poverty_rate_change:+.3f} pp ({poverty_percent_change:+.2f}%)")
print(f"")
print(f"Child Poverty Rate:")
print(f"  Baseline: {child_poverty_baseline_rate:.2f}%")
print(f"  Reform: {child_poverty_reform_rate:.2f}%")
print(f"  Change: {child_poverty_rate_change:+.3f} pp ({child_poverty_percent_change:+.2f}%)")
print("="*70)


POVERTY IMPACT
Overall Poverty Rate:
  Baseline: 21.52%
  Reform: 21.28%
  Change: -0.232 pp (-1.08%)

Child Poverty Rate:
  Baseline: 23.23%
  Reform: 22.69%
  Change: -0.538 pp (-2.32%)


In [15]:
# Income bracket analysis (matching microsimulation.py logic)
income_brackets = [
    (0, 50000, "Under $50k"),
    (50000, 100000, "$50k-$100k"),
    (100000, 150000, "$100k-$150k"),
    (150000, 200000, "$150k-$200k"),
    (200000, float('inf'), "Over $200k")
]

bracket_data = []
for min_income, max_income, label in income_brackets:
    # Include ALL households affected (positive or negative impact)
    mask = (agi >= min_income) & (agi < max_income) & (np.abs(net_income_change) > 1)
    bracket_affected = mask.sum()  # Unweighted count
    bracket_cost = net_income_change[mask].sum()  # Weighted sum (MicroSeries)
    bracket_avg = net_income_change[mask].mean() if bracket_affected > 0 else 0  # Weighted mean
    
    bracket_data.append({
        "Income Bracket": label,
        "Households (sample)": f"{bracket_affected:,.0f}",
        "Total Impact": f"${bracket_cost:,.0f}",
        "Avg Benefit": f"${bracket_avg:,.0f}"
    })

bracket_df = pd.DataFrame(bracket_data)

print("\n" + "="*70)
print("IMPACT BY INCOME BRACKET")
print("="*70)
print(bracket_df.to_string(index=False))
print("="*70)


IMPACT BY INCOME BRACKET
Income Bracket Households (sample) Total Impact Avg Benefit
    Under $50k              40,426  $21,436,112        $530
    $50k-$100k              28,332   $7,949,591        $281
   $100k-$150k              25,009   $5,345,995        $214
   $150k-$200k              12,540   $2,412,671        $192
    Over $200k              13,179   $1,981,570        $150


In [16]:
# Save summary
summary_data = {
    'Metric': [
        'Dataset',
        'Year',
        'Total Cost',
        'Beneficiary Households (sample)',
        'Average Benefit',
        'Children Affected',
        'Winners',
        'Winners Rate',
        'Losers',
        'Losers Rate',
        'Total Households',
        'Poverty Rate (Baseline)',
        'Poverty Rate (Reform)',
        'Poverty Rate Change (pp)',
        'Poverty Percent Change',
        'Child Poverty Rate (Baseline)',
        'Child Poverty Rate (Reform)',
        'Child Poverty Rate Change (pp)',
        'Child Poverty Percent Change'
    ],
    'Value': [
        'RI-2',
        str(YEAR),
        f"${total_cost:,.0f}",
        f"{beneficiaries:,.0f}",
        f"${avg_benefit:,.0f}",
        f"{children_affected:,.0f}",
        f"{winners:,.0f}",
        f"{winners_rate:.1f}%",
        f"{losers:,.0f}",
        f"{losers_rate:.1f}%",
        f"{total_households:,.0f}",
        f"{poverty_baseline_rate:.2f}%",
        f"{poverty_reform_rate:.2f}%",
        f"{poverty_rate_change:+.3f}",
        f"{poverty_percent_change:+.2f}%",
        f"{child_poverty_baseline_rate:.2f}%",
        f"{child_poverty_reform_rate:.2f}%",
        f"{child_poverty_rate_change:+.3f}",
        f"{child_poverty_percent_change:+.2f}%"
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('governor_proposal_ri2_results.csv', index=False)
print("\nResults saved to: governor_proposal_ri2_results.csv")
print(summary_df.to_string(index=False))


Results saved to: governor_proposal_ri2_results.csv
                         Metric       Value
                        Dataset        RI-2
                           Year        2027
                     Total Cost $39,879,203
Beneficiary Households (sample)     120,966
                Average Benefit        $330
              Children Affected     221,871
                        Winners   7,144,727
                   Winners Rate       31.3%
                         Losers           0
                    Losers Rate        0.0%
               Total Households  22,795,617
        Poverty Rate (Baseline)      21.52%
          Poverty Rate (Reform)      21.28%
       Poverty Rate Change (pp)      -0.232
         Poverty Percent Change      -1.08%
  Child Poverty Rate (Baseline)      23.23%
    Child Poverty Rate (Reform)      22.69%
 Child Poverty Rate Change (pp)      -0.538
   Child Poverty Percent Change      -2.32%
